In [13]:
from pwn import *
import sys, ecdsa, hashlib
sys.path.append('/tmp/challenges/tentacool/lattice-based-cryptanalysis')

from lbc_toolkit import hnp

E = ecdsa.curves.SECP256k1
g = E.generator * 246
n = Integer(E.order)
p = E.curve.p()

def hex_pad(m):
    m = hex(m)[2:]
    if len(m) % 2 == 1:
        m = m.zfill(len(m) + 1)
    return m

def sign_message(m):
    io.sendlineafter(b">> ", b"2")
    io.sendlineafter(b"Message: ", m.encode())
    
    io.recvuntil(b"Signature: ")
    e, s = [int(num) for num in io.recvline().decode()[1:-2].split(", ")]

    return e, s

def local_sign(x, m: bytes):
    k = ecdsa.rfc6979.generate_k(g.x(), x % g.x(), hashlib.sha512, m) * p % n
    r = g * k
    r = r.x()
    e = int(hashlib.sha256(str(r).encode() + m).hexdigest(), 16)
    s = (k + x * e) % n
    return e, s

# def construct_matrix(es, ss):
#     M_size = len(es)
#     M = identity_matrix(QQ, M_size)
#     M = M.stack(vector([es[i]*pow(p,-1,n) for i in range(M_size)]))
#     M = M.stack(vector([-ss[i]*pow(p,-1,n) for i in range(M_size)]))
#     M = M.augment(identity_matrix(M_size + 2))
    
#     for i in range(M_size):
#         row = [n if i == j else 0 for j in range(2*M_size + 2)]
#         M = M.stack(vector(row)) 
#     return M

io = process(["python3", "/tmp/challenges/tentacool/chall.py"])
context.log_level = "info"

es, ss = [], []
for i in range(25):
    e, s = sign_message(hex_pad(i+128))
    es.append(-(e * pow(p, -1, n) % n))
    ss.append(-(s * pow(p, -1, n) % n))

x = hnp(n, es, ss, 2**245, verbose=True)
print("x =", x)

target = b"Tentacool approves of this message!"
e, s = local_sign(x, target)
io.sendlineafter(b">> ", b"3")
io.sendlineafter(b"Message: ", target.hex())
io.sendlineafter(b"e: ", hex_pad(e).encode())
io.sendlineafter(b"s: ", hex_pad(s).encode())

print(io.recv())

[x] Starting local process '/home/sage/sage/local/var/lib/sage/venv-python3.12.5/bin/python3'
[+] Starting local process '/home/sage/sage/local/var/lib/sage/venv-python3.12.5/bin/python3': pid 426


[hnp] Lattice dimensions: (27, 27)
[hnp] Lattice reduction took 0.051s
x = 113968943347999331551472773340652091459321512007990293248765913978853811147489
b"Valid\nb'chrono{63effc06a083538135d86293e8f9cc1b}'\n"


/tmp/ipykernel_244/3908340964.py:62: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  io.sendlineafter(b"Message: ", target.hex())
